## README: Making input fasta files

This script wrangles multiple sequence alignments and fasta files from `Nextstrain/fauna` to make fasta files of sequences that we want to include specifically in our analysis of Zika in the Americas. A variety of fasta files are outputted, including ones that can be read in to BEAST, and ones that can be read in to `Nextstrain/augur`.

In [87]:
#### import libraries ####
from Bio import SeqIO
from Bio import AlignIO
import datetime

date = datetime.datetime.now().strftime ("%Y-%m-%d")


In [88]:
#### infile paths #### 
zika_msa_stripped = "/Users/alliblk/Desktop/gitrepos/augur/zika/processed/zika_aligned_stripped.mfa"
fauna_file = "/Users/alliblk/Desktop/gitrepos/fauna/data/zika.fasta"

#### outfile paths #### 
americas_file = '/Users/alliblk/Desktop/gitrepos/zika-usvi/data/fastas/american-zika-{}.fasta'.format(date)
americas_frenchpol_file = '/Users/alliblk/Desktop/gitrepos/zika-usvi/data/fastas/american-frenchPolyn-zika-{}.fasta'.format(date)

usvi_file = "/Users/alliblk/Desktop/gitrepos/zika-usvi/data/fastas/usvi-{}.fasta".format(date)
usvi_primary_clade_file = '/Users/alliblk/Desktop/gitrepos/zika-usvi/data/fastas/usvi-primary-clade-{}.fasta'.format(date)


In [89]:
# Sequences that should be removed from the analysis alignment #
# (reasons for removal are noted in the comments) #

#### geographic exclusion criteria #### 
regions_to_exclude1 = ['southeast_asia', 'oceania', 'japan_korea', 'china','europe'] #french polynesia out
regions_to_exclude2 = ['southeast_asia', 'japan_korea', 'china','europe'] #french polynesia in

#### sequence characteristic exclusion criteria (based on Augur processing) #### 
drop_for_indel = ["CX17"] #sequence has large number of indels.
drop_for_contamination = ['ZF36_36S'] #possible contamination.
drop_duplicates = ["Dominican_Republic/2016/PD2", "GD01", "GDZ16001", "VEN/UF_2/2016"] #true strains, but duplicates of other strains
#excessive terminal branch length, likely indicative of large amount of sequencing error.
drop_for_excessive_terminal_branch_length = ["Bahia04", "JAM/2016/WI_JM6", "Bahia11", "Bahia12", "DOM/2016/MA_WGS16_009", "VE_Ganxian", "BRA/2016/FC_DQ60D1", "CX5"]
drop_unknown_export = ["VR10599/Pavia/2016", "34997/Pavia/2016"] # travel cases where country of infection acquisition is unknown
drop_molecClock_off = ["THA/PLCal_ZV/2013", "SK403/13AS", "SV0010/15", "SK364/13AS"] #outliers on root to tip analyses. Aren't adhering to molecular clock.
drop_augurPruned = ['Bahia05', 'Bahia15'] #An augur run will spit this out as verbose output, and you can also tell based on what sequences are included in the JSONs

In [90]:
# Get stats on how many of the Americas fauna sequences are dropped by augur because they are problematic.

seqs_in_americas_count = 0
indel_drop_count = 0
contam_drop_count = 0
duplicate_drop_count = 0
term_branch_leng_drop_count = 0
export_drop_count = 0
clock_drop_count = 0
augur_pruned_count = 0 

for key in strain_header_dict.keys():
    if strain_header_dict[key].split('|')[4] in regions_to_exclude1: #only look at Americas sequences
        continue
    else:
        seqs_in_americas_count += 1
        if key in drop_for_indel:
            indel_drop_count += 1
        elif key in drop_for_contamination:
            contam_drop_count += 1
        elif key in drop_duplicates:
            duplicate_drop_count += 1
        elif key in drop_for_excessive_terminal_branch_length:
            term_branch_leng_drop_count += 1
        elif key in drop_unknown_export:
            export_drop_count += 1
        elif key in drop_molecClock_off:
            clock_drop_count += 1
        elif key in drop_augurPruned:
            augur_pruned_count += 1

print "There are {} Zika sequences available from fauna".format(len(strain_header_dict.keys()))
print "There are {} American Zika sequences available from fauna".format(seqs_in_americas_count)
print "Augur removed {} sequence(s) sampled from the Americas because they contain excessive indels".format(indel_drop_count)
print "Augur removed {} sequence(s) sampled from the Americas because they show evidence of contamination".format(contam_drop_count)
print "Augur removed {} sequence(s) sampled from the Americas because they are duplicate strains".format(duplicate_drop_count)
print "Augur removed {} sequence(s) sampled from the Americas because they demonstrate excessive terminal branch length".format(term_branch_leng_drop_count)
print "Augur removed {} sequence(s) sampled from the Americas because they are exported cases where country of infection acquistion could not be determined".format(export_drop_count)
print "Augur removed {} sequence(s) sampled from the Americas because they did not follow the molecular clock".format(clock_drop_count)
print "Alli removed {} additional sequence(s) because they outliers/poor quality/generally problematic".format(augur_pruned_count)

There are 501 Zika sequences available from fauna
There are 335 American Zika sequences available from fauna
Augur removed 1 sequence(s) sampled from the Americas because they contain excessive indels
Augur removed 0 sequence(s) sampled from the Americas because they show evidence of contamination
Augur removed 3 sequence(s) sampled from the Americas because they are duplicate strains
Augur removed 8 sequence(s) sampled from the Americas because they demonstrate excessive terminal branch length
Augur removed 0 sequence(s) sampled from the Americas because they are exported cases where country of infection acquistion could not be determined
Augur removed 0 sequence(s) sampled from the Americas because they did not follow the molecular clock
Alli removed 2 additional sequence(s) because they outliers/poor quality/generally problematic


In [100]:
# make dicts holding data in different structures

# dict that will allow matching of fauna header to MSA header
with open(fauna_file,'rU') as file:
    strain_header_dict={line.split('|')[0].replace('>',''):line.strip() for line in file if line.startswith('>')}

# dict that will allow matching of fauna header to fauna sequnces (need this for making augur-formatted infiles)
fauna_dict = SeqIO.to_dict(SeqIO.parse(fauna_file, 'fasta'))
print len(fauna_dict)

# dict that has only strain name as key, but value is aligned, stripped to reference sequence
zika_msa = AlignIO.read(open(zika_msa_stripped),'fasta')
zika_msa_dict = {record.id:record.seq for record in zika_msa}

#samples that Augur prunes during tree building, therefore the genomes are still in the MSA and need to be removed
zika_msa_dict_pruned = {key:value for key,value in zika_msa_dict.items() if key not in drop_augurPruned}
print len(zika_msa_dict_pruned)

### NOTE TO SELF:
#now that I'm doing all the trouble shooting in Augur, the sequence set that I want will probably be 
#the input to Augur, IE I probably won't need a pruned MSA dict in the future.


#check that you in fact have the trimmed down alignment loaded in.
for key in zika_msa_dict_pruned.keys():
    assert len(zika_msa_dict_pruned[key]) == 10769

501
300


In [121]:
#trouble shooting some samples that are slightly poorer quality. Trying to see what propotion of the genome is N 
# for some samples that are looking a bit odd in the tree.
naccache_dict = {key:value for key,value in fauna_dict.items() if key.startswith('Bahia')}
print naccache_dict.keys()
n_counts = {}
non_n_counts = {}

for key in naccache_dict.keys():
    n_count = 0
    seq_len = len(naccache_dict[key].seq)
    for base in naccache_dict[key].seq:
        if base == 'n':
            n_count +=1
        else:
            continue
    n_counts[key.split('|')[0]] = n_count
    non_n_counts[key.split('|')[0]] = seq_len-n_count
    #n_counts[key] = n_count
print n_counts
print non_n_counts

['Bahia09|zika|KU940224|2015-08-01|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia04|zika|KX101062|2015-06-XX|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia07|zika|KU940228|2015-07-01|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia15|zika|KX101065|2016-01-XX|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia12|zika|KX101067|2015-05-XX|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia02|zika|KX101060|2015-05-XX|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia01|zika|KX101066|2015-05-XX|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia03|zika|KX101061|2015-05-XX|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia08|zika|KU940227|2015-07-15|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia11|zika|KX101064|2015-04-XX|south_america|brazil|brazil|brazil|genbank|genome|Naccache', 'Bahia05|zika|KX101063|2015-12-XX|south_america|b

## Making fasta files formatted for input into Augur pipeline.

Since Augur is a faster tool for building trees and doing ancestral state reconstruction than BEAST, I'm troubleshooting possible issues with the input alignment and looking for outliers (general dataset QC) via Augur builds. Augur input fasta files need to be formatted exactly the same was as the fauna output fasta file. So what I'm doing here is keeping header formatting the same as fauna, but subsampling down to exclude genomes from countries that I don't want to include, or samples that appear to be outliers etc.

In [92]:
# prune dicts geographically
# note that all of the samples that should be dropped except those specified in drop_augurPruned are 
# hardcoded into Augur as samples that should be dropped.
# therefore they do not need to be dropped here in this script
# I've entered them above mainly so I can keep stats on numbers of samples getting dropped and why
geoPruned_fauna_dict_americasOnly = {key:value for key,value in fauna_dict.items() if key.split('|')[4] not in regions_to_exclude1 and key.split('|')[0] not in drop_augurPruned}
geoPruned_fauna_dict_includeOceania = {key:value for key,value in fauna_dict.items() if key.split('|')[4] not in regions_to_exclude2 and key.split('|')[0] not in drop_augurPruned}

#print out fauna-format files for input into augur

# Americas only
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/fastas/augur-americas-only.fasta','w') as file:
    for key in geoPruned_fauna_dict_americasOnly.keys():
        file.write(str('>' + geoPruned_fauna_dict_americasOnly[key].description + '\n' + geoPruned_fauna_dict_americasOnly[key].seq + '\n'))
        
#Americas + french polynesia
with open('/Users/alliblk/Desktop/gitrepos/zika-usvi/data/fastas/augur-americas-andfp.fasta','w') as file:
    for key in geoPruned_fauna_dict_includeOceania.keys():
        file.write(str('>' + geoPruned_fauna_dict_includeOceania[key].description + '\n' + geoPruned_fauna_dict_includeOceania[key].seq + '\n'))
        

## Combining the augur processed multiple sequence alignment with the fauna-output fasta

Here I want to combine attributes of both the `Nextstrain/augur` processed Zika MSA with the fasta output from `Nextstrain/fauna`. The Fauna download has the strain information in the desired fasta format, with all necessary metadata (sampling date, geography) in the header. The processed multiple sequence alignment however has been aligned with mafft and stripped to the WHO ZIKV reference genome, and therefore represents the sequence alignment that we want.

The header from the MSA contains the strain name of the sample, which is also in the fauna header. Therefore I will use key matching to make a new fasta file that combines the header from the fauna file with the sequences from the augur msa. The fauna header will be trimmed down when writing out the fastas to remove superfluous information in the header and to ensure that the headers can be read in to FigTree.

In [93]:
output_dict = {}
for key in zika_msa_dict_pruned.keys():
    header = strain_header_dict[key]
    seq = zika_msa_dict_pruned[key]
    output_dict[header] = seq

americas_count = 0
oceania_count = 0
non_americas_non_oceania_count = 0 

for key in output_dict.keys():
    if key.split('|')[4] not in regions_to_exclude1:
        americas_count += 1
    elif key.split('|')[4] == 'oceania':
        oceania_count += 1
    elif key.split('|')[4] in regions_to_exclude1:
        non_americas_non_oceania_count += 1

print len(output_dict.keys())
print americas_count
print oceania_count
print non_americas_non_oceania_count

325
299
26
0


In [94]:
#print out Americas only multiple sequence alignment
with open(americas_file,'w') as out_file:
    for key in output_dict.keys():
        if key.split('|')[4] in regions_to_exclude1:
            continue
        else:
            split_name = key.split('|')
            header = split_name[0] +'|'+ split_name[3] + '|'+ split_name[4] + '|'+ split_name[5]
            out_file.write(str(header + '\n' + output_dict[key] + '\n'))

# Uncomment to print out alignment that still contains french polynesian sequences
#print out Americas and french polynesia multiple sequence alignment
'''
with open(americas_frenchpol_file,'w') as out_file:
    for key in output_dict.keys():
        if key.split('|')[4] in regions_to_exclude2:
            continue
        else:
            split_name = key.split('|')
            header = split_name[0] +'|'+ split_name[3] + '|'+ split_name[4] + '|'+ split_name[5] 
            out_file.write(str(header + '\n' + output_dict[key] + '\n'))
'''
#print out USVI only multiple sequence alignment
with open(usvi_file,'w') as out_file:
    for key in output_dict.keys():
        if key.split('|')[5] == 'usvi':
            split_name = key.split('|')
            header = split_name[0] +'|'+ split_name[3] + '|'+ split_name[4] + '|'+ split_name[5]
            out_file.write(str(header + '\n' + output_dict[key] + '\n'))
        else:
            continue
        